In [1]:
import numpy as np
import tensorflow as tf

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

from __future__ import print_function

from keras.preprocessing import sequence
from keras.datasets import imdb

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = tf.keras.models.Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(tf.keras.layers.Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(tf.keras.layers.Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(tf.keras.layers.Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(tf.keras.layers.GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(tf.keras.layers.Dense(hidden_dims))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

newmodel = tf.keras.estimator.model_to_estimator(keras_model = model)

model.input_names

/home/wh52/.pyenv/versions/anaconda3-5.0.0/envs/gpu_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)
Build model...
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp5safkivr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f085db8c470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


['embedding_1_input']

In [3]:
y_train = y_train.reshape(-1, 1)


train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {'embedding_1_input':x_train},
    y = y_train,num_epochs=3,
    shuffle=False)

newmodel.train(input_fn=train_input_fn)

y_test = y_test.reshape(-1, 1)
test_input = tf.estimator.inputs.numpy_input_fn(x = {'embedding_1_input':x_test},
                                               y = y_test,
                                               batch_size = batch_size,
                                               shuffle=False)
e = newmodel.evaluate(test_input)
print("test accuracy:", e['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp5safkivr/model.ckpt-587
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 588 into /tmp/tmp5safkivr/model.ckpt.
INFO:tensorflow:loss = 0.39129928, step = 588
INFO:tensorflow:global_step/sec: 45.4887
INFO:tensorflow:loss = 0.34778014, step = 687 (2.199 sec)
INFO:tensorflow:global_step/sec: 58.0666
INFO:tensorflow:loss = 0.38180655, step = 787 (1.723 sec)
INFO:tensorflow:global_step/sec: 54.884
INFO:tensorflow:loss = 0.30987516, step = 887 (1.822 sec)
INFO:tensorflow:global_step/sec: 53.6236
INFO:tensorflow:loss = 0.2830492, step = 987 (1.866 sec)
INFO:tensorflow:global_step/sec: 50.8794
INFO:tensorflow:loss = 0.31849274, step = 1087 (1.965 sec)
INFO:tensorflow:Saving checkpoints for 1174 into /tmp/tmp5safkivr/model.